Run in Azure ML. 

Below code is to download and unpack the dataset zip from Kaggle.com

In [ ]:
import os
import zipfile
# name of the zip file you want to unzip
local_zip = 'coronahack-respiratory-sound-dataset.zip'
# opening a file with mode parameter 'r' : read existing file
zip_ref = zipfile.ZipFile(local_zip, 'r')
# extract all contents of the zip file
zip_ref.extractall('')
# close the file
zip_ref.close()

Install Librosa - audio analsis package. 


In [ ]:
!pip install librosa

Pre-Processing of Coronahack Respiratory Sound Dataset below... 

In [ ]:
import math
import os
import librosa
import json

training_dataset_path = 'CoronaHack-Respiratory-Sound-Dataset/data/train'
SAMPLE_RATE = 22050
DURATION = 30  # in seconds
SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

numberOfSegments = 10
hopLength = 512
n_MFCC = 13
n_FFT = 2048

data = {
    "mapping": [
        'Healthy',
        'Astma',
        'Covid',
        'Astma-Covid',
        'inconclusive'
    ],
    "mfcc": [],
    "labels": [],
}


def getLabel(metadata):
    label = 0
    try:
        if metadata['asthma'] == 'True':
            # has asthma
            if metadata['covid_status'] == 'healthy':
                print('asthma')
                return 1
            print('asthma + covid')
            return 3
    except:
        label = 0
    try:
        if metadata['covid_status'] == 'healthy':
            return 0
        else:
            return 1
    except:
        return 4


for i, (dirpath, _, filenames) in enumerate(os.walk(training_dataset_path)):
    if dirpath == training_dataset_path:
        continue

    if len(filenames) == 0:
        continue

    metadata = {}

    for file in filenames:
        path = os.path.join(dirpath, file)

        if str(file) == 'metadata.json':
            f = open(path)
            metadata = json.load(f)

            label = getLabel(metadata)
            data["labels"].append(label)

            continue

        if str(file) == 'breathing-deep.wav':
            # load audio
            try:
                signal, sampleRate = librosa.load(path, sr=SAMPLE_RATE)
            except:
                print("Error processing")
                continue

            # get MFCC
            MFCC = librosa.feature.mfcc(
                y=signal,
                sr=sampleRate,
                n_fft=n_FFT,
                n_mfcc=n_MFCC,
                hop_length=hopLength).T

            # store mfcc for segment
            data["mfcc"].append(MFCC.tolist())



Just some output to see number of MFCC created during pre-processing. 

In [ ]:
# number of mfcc from data

count = 0
for i in range(0, len(data["mfcc"])):
    for j in range(0, len(data["mfcc"][i])):
        for mffcc in data["mfcc"][i][j]:
            count += 1

print("number of MFCCs: {}".format(count))
